# Wine Quality Binary Classification with Logistic Regression and SVM

#### Project Overview
The purpose of this project is to classify red wines as either high-quality or low-quality based on their chemical properties, including 11 physicochemical measurements and their quality rating from professional tasters. We frame the task as a binary classification problem, labeling wines as high-quality if their rating is 6.5 or above and low-quality if their rating is below 6.5. 

#### Motivation
Though wine quality is subjective, it's a commercially important property and often requires costly sensory tests to identify. Predictive modeling using physicochemical measurements can provide an efficient and cost-effective way to identify wine quality.

#### Dataset Information
The dataset, sourced from the UCI Machine Learning Repository, reflects the rarity of high-quality wine, with high-quality wines constituting only ~7% of the data. The analysis works with the imbalanced nature of the data, demonstrating model optimization in both the logistic regression and support vector machine models used, enabling them to perform as effectively as possible given the task. 

#### Process Overview
We begin by establishing a logistic regression baseline and optimizing its performance through threshold tuning and appropriate class weighting. We then transition to support vector machines (SVM) with various kernels and thresholds to better accommodate and capture any potential non-linear class boundaries.

#### Project Outcome
The focus of this project is to maximize the recall of the quality wine class and the precision with which the machine identifies quality wine. It demonstrates a complete ML pipeline, including preprocessing, model selection, class imbalance accommodation, metric optimization, and analysis. The final models show comprehensive handling of the imbalanced classification task, providing a solid basis for further exploration with more advanced or ensemble classifiers.

### License

This notebook is licensed under the MIT License. See the LICENSE file for details.

### Library Imports

In [437]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve
import statsmodels.api as sm
from ISLP import confusion_table

### Loading Data

In [438]:
df = pd.read_csv("winequality-red.csv")
df[:5]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [439]:
df.shape

(1599, 12)

### Preprocessing

High-quality wines are labeled as 1, and low-quality wines are labeled as 0. Furthermore, we want a train-test split of approximately 80% train and 20% test: <br>
1599 * .8 = 1280 train<br>
1599-1280 = 319 test

In [440]:
# Here we return a boolean series describing every element in the 'quality' column
# astype(int) converts the T/F to 1/0
df['quality'] = (df['quality'] >= 6.5).astype(int)

In [441]:
# Dropping our output column to make the x values
X = df.drop(['quality'], axis = 1) # axis = 1 denotes that we're dropping a column and not a row
y = df['quality']

scaler = StandardScaler() # Scaler from sklearn to ensure every predictor has a proportionate effect
# Once we scale it, it becomes a NumPy array
X_scaled_array = scaler.fit_transform(X)
# Here we turn the NumPy array back into a Pandas df 
# syntax pd.DataFrame(array, column names)
X_scaled = pd.DataFrame(X_scaled_array, columns = X.columns)
X_scaled_w_intercept = sm.add_constant(X_scaled) # Creating a new dataframe with the intercepts because sm does not do so automatically

train = (X_scaled_w_intercept.index <= 1280)
X_train = X_scaled_w_intercept.loc[train]
X_test = X_scaled_w_intercept.loc[~train]
y_train = y.loc[train]
y_test = y.loc[~train]

### Logistic Regression

We begin with logistic regression to evaluate the effectiveness of a computationally conservative and relatively inflexible model on the data before moving on to more complex tools.

#### Preliminary Fit

In [442]:
logreg = sm.GLM(y_train,
                 X_train,
                 family = sm.families.Binomial())
results = logreg.fit()
summarize(results)

,coef,std err,z,P>|z|
const,-2.6756,0.148,-18.114,0.000
fixed acidity,0.5592,0.253,2.208,0.027
volatile acidity,-0.3720,0.148,-2.514,0.012
citric acid,0.1766,0.180,0.980,0.327
residual sugar,0.4071,0.114,3.572,0.000
chlorides,-0.3404,0.159,-2.143,0.032
free sulfur dioxide,0.0694,0.138,0.502,0.616
total sulfur dioxide,-0.5204,0.167,-3.108,0.002
density,-0.6199,0.231,-2.686,0.007
pH,0.1755,0.169,1.038,0.299


In [443]:
probs = results.predict(X_test)
probs[:10]

1281    0.058631
1282    0.061735
1283    0.025776
1284    0.119657
1285    0.115439
1286    0.592464
1287    0.407059
1288    0.068596
1289    0.068596
1290    0.064801
dtype: float64

In [444]:
probs.shape

(318,)

In [445]:
labels = np.array([0] * 318) # Making an array of all 0 labels
# Using a boolean mask here with probs >= .5
labels[probs >= .5] = 1 # Changing elements of labels to 1 if the corresponding element in probs is >= .5
confusion_t = confusion_table(labels, y_test)
confusion_t

Truth,0,1
Predicted,,
0,288,20
1,6,4


In [446]:
np.mean(labels == y_test), np.mean(labels != y_test)

(0.9182389937106918, 0.08176100628930817)

Although the model's accuracy of 91% seems satisfactory at first glance, the model could have achieved a ~93% accuracy by simply labeling everything as low-quality wine. When the true outcome was 0, the model predicted correctly 288/294 times, yielding a recall rate of ~ 97% for that class. However, when the true outcome was 1, the model predicted correctly 4/24 times, yielding a recall rate of ~ 17% for that class. The latter is very poor, so we'll have to adjust our approach. This performance is likely due to the class imbalance in the training data, so we'll try lowering the threshold first.

It's more important to identify quality wine, even if the prediction is wrong occasionally. False negatives are more harmful than false positives here, so we'll choose the decision boundary to maximize recall and precision for class 1.

Confusion table indexing for convenience:
True negative [0][0], false positive [0][1], false negative [1][0], true positive [1][1]

#### Manual Threshold Tuning

In [447]:
# Making class 1 recall rate and precision functions for ease of use
def c1_recall(confusion_t):
    return confusion_t[1][1]/(confusion_t[1][1] + confusion_t[1][0])

def m_precision(confusion_t):
    return confusion_t[1][1]/(confusion_t[1][1] + confusion_t[0][1])

In [448]:
# Creating a for loop function to evaluate different thresholds by hand
thresholds = np.arange(0.4, 0.0, -0.1) # Starts at .4 and ends at .1, incrementing by -.1 each time

for i in thresholds:
    labels = np.array([0] * 318)
    labels[probs >= i] = 1
    confusion_t = confusion_table(labels, y_test)
    recall_rate = c1_recall(confusion_t)
    precision = m_precision(confusion_t)
    print("Threshold: ", round(i, 3), " Class 1 recall: ", round(recall_rate, 3), "  Precision: ", round(precision, 3))

Threshold:  0.4  Class 1 recall:  0.333   Precision:  0.364
Threshold:  0.3  Class 1 recall:  0.5   Precision:  0.308
Threshold:  0.2  Class 1 recall:  0.708   Precision:  0.293
Threshold:  0.1  Class 1 recall:  0.792   Precision:  0.173


#### Class Balancing

These results are poor, so we'll have to adjust the model instead. We'll try switching to sklearn to easily use class weighting as stats models doesn't support class weights in GLM.

In [449]:
logreg = LogisticRegression(class_weight = 'balanced')
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict_proba(X_test)
logreg_pred[:5]

array([[0.77716222, 0.22283778],
       [0.71733582, 0.28266418],
       [0.8827808 , 0.1172192 ],
       [0.59246669, 0.40753331],
       [0.57800344, 0.42199656]])

In [450]:
logreg_labels = np.where(logreg_pred[:, 1] >= .5, 1, 0)
confusion_t = confusion_table(logreg_labels, y_test)
confusion_t
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.75
Model Precision:  0.23376623376623376


#### F1 Threshold Tuning

The model precision is higher, but the recall has dropped off compared to the last model/decision boundary combination. Just in case, we'll try out different boundaries using the F1 score to ensure we get the best recall-precision trade-off possible for this model instance.

In [451]:
# Here we multiply the arrays element by element to get a new array using the F1 formula directly
# 1e-9 is used to prevent division by zero
def calc_f1(precision, recall):
    return (2 * (precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-9))

In [452]:
# The precision_recall_curve returns the following in order: precision, recall, and threshold as NumPy arrays
# Syntax: precision_recall_curve(true outcomes, prob estimated by machine)
precision, recall, threshold = precision_recall_curve(y_test, logreg_labels)

f1_scores = calc_f1(precision, recall)
best_index = np.argmax(f1_scores) # Returns the index of the largest value
best_threshold = thresholds[best_index]
best_f1 = f1_scores[best_index]
best_precision = precision[best_index]
best_recall = recall[best_index]

print("Best threshold: ", best_threshold)
print("Best precision: ", best_precision)
print("Best recall: ", best_recall)

Best threshold:  0.30000000000000004
Best precision:  0.23376623376623376
Best recall:  0.75


#### Feature Selection

The recall value is good but the precision is still very poor. Thus, let's switch gears and reconstruct the model, excluding one of the predictors which showed high p values in our initial fitting to see if its influence is disrupting the performance of the model. 

In [453]:
# df variable short for "no citric acid df"
n_ca_df = X_scaled_w_intercept.drop(['citric acid'], axis = 1)

n_ca_df_train = n_ca_df[train]
n_ca_df_test = n_ca_df[~train]
# y_train and y_test will remain the same

In [454]:
# We'll use sklearn again with balanced class weights, as it increased the precision last time
logreg.fit(n_ca_df_train, y_train)
logreg_pred = logreg.predict_proba(n_ca_df_test)

In [455]:
logreg_labels = ([0] * 318)
logreg_labels = np.where(logreg_pred[:, 1] >= .5, 1, 0)
confusion_t = confusion_table(logreg_labels, y_test)
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.75
Model Precision:  0.23684210526315788


Excluding the citric acid predictor had a negligible effect on model performance. Further exclusion of predictors with high p-values is unlikely to yield constructive results due to the limited model framework. Given the poor performance of logistic regression, even with model optimization, we'll retain the full feature set for comparison with more flexible models. Transitioning to a Support Vector Machine (SVM) represents the next logical step as it offers greater modeling flexibility and maintains reasonable computational demands. It's likely that the class boundaries in this dataset aren't linearly separable, contributing to the ineffectiveness of logistic regression. SVMs can model more flexible, non-linear decision boundaries, allowing them to capture more of the complexities of the dataset.

### SVM

#### Preliminary Fit with RBF Kernel

In [456]:
# We already have a scaled dfs with an intercept, X_train/X_test, from the logistic regression model preparation

# kernel = 'rbf' maps features into a higher-dimensional space, allowing for non-linear decision boundaries
# probability = True tells the model to also compute probabilities so we can threshold tune
# In addition, we balance the class weight to adjust for the class imbalance
# random_state = 42 ensures reproducibility
svm_model = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced', random_state = 0)
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced', probability=True, random_state=0)

We'll evaluate the plain model before adjusting the decision boundary.

In [457]:
svm_probs = svm_model.predict_proba(X_test)
svm_labels = np.where(svm_probs[:, 1] >= 0.5, 1, 0)
confusion_t = confusion_table(svm_labels, y_test)
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.3333333333333333
Model Precision:  0.42105263157894735


The results indicate that the current SVM correctly identifies 1s only 1/3 of the time, and of the predicted 1s, only about 42% were true 1s. The data is imbalanced (despite class_weight = 'balanced'), so the machine may be too conservative when predicting class 1. As discussed before, for our purposes, it is better to identify more quality wine with a few mistakes than to misidentify good wine as bad. Thus, we'll adjust the threshold using F1 values.

#### F1 Threshold Tuning

In [458]:
probs = svm_probs[:, 1] # Getting the class 1 probabilities
precision, recall, thresholds = precision_recall_curve(y_test, probs)
# Performs operations element-wise (excluding the last element) and returns an array of F1 scores
f1_scores = calc_f1(precision, recall)
best_index = np.argmax(f1_scores)
best_threshold = thresholds[best_index]
print("Best threshold: ", best_threshold)

Best threshold:  0.35932779933890735


In [459]:
# Now let's apply the best threshold
svm_labels = np.where(svm_probs[:, 1] >= best_threshold, 1, 0)
confusion_t = confusion_table(svm_labels, y_test)
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.5833333333333334
Model Precision:  0.4


Adjusting the threshold improved the model significantly, keeping the precision around 40% while increasing the recall to almost 60%. Already, the SVM model is performing much better than logistic regression. Let's perform some model optimization to allow the SVM to perform as well as possible on the dataset. We'll try a degree-2 polynomial kernel first, then a degree-3.

#### Polynomial Kernel Tuning

In [460]:
# Implementing with degree 2 first to avoid overfitting
svm_model = SVC(kernel = 'poly', degree = 2, probability = True, class_weight = 'balanced', random_state = 0)
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced', degree=2, kernel='poly', probability=True,
    random_state=0)

In [461]:
svm_poly_probs = svm_model.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, svm_poly_probs)

f1_scores = calc_f1(precision, recall)
best_index = np.argmax(f1_scores)
best_threshold = thresholds[best_index]
print("Best threshold: ", best_threshold)

Best threshold:  0.330748532479003


In [462]:
# Now let's apply the best threshold
svm_poly_labels = np.where(svm_poly_probs >= best_threshold, 1, 0)
confusion_t = confusion_table(svm_poly_labels, y_test)
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.5833333333333334
Model Precision:  0.45161290322580644


The polynomial kernel raises the precision by more than 5% while leaving the recall the same. This is a modest improvement, so let's try a degree of 3 to see if more flexibility enhances the performance.

In [463]:
svm_model = SVC(kernel = 'poly', degree = 3, probability = True, class_weight = 'balanced', random_state = 0)
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced', kernel='poly', probability=True, random_state=0)

In [464]:
svm_poly_probs = svm_model.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, svm_poly_probs)

f1_scores = calc_f1(precision, recall)
best_index = np.argmax(f1_scores)
best_threshold = thresholds[best_index]
print("Best threshold: ", best_threshold)

Best threshold:  0.2052551333083447


In [465]:
# Now let's apply the best threshold
svm_poly_labels = np.where(svm_poly_probs >= best_threshold, 1, 0)
confusion_t = confusion_table(svm_poly_labels, y_test)
print("Class 1 recall: ", c1_recall(confusion_t))
print("Model Precision: ", m_precision(confusion_t))

Class 1 recall:  0.5833333333333334
Model Precision:  0.3783783783783784


Increasing the degree decreased the model's precision, hindering its performance. Out of all the models tested thus far, the support vector machine with a degree-2 polynomial kernel and optimized decision threshold performs best in terms of recall and precision.

### Evaluation and Conclusions

This project utilized logistic regression and support vector machines to evaluate the quality of wine based on its physicochemical properties. Logistic regression was chosen as a baseline model due to its interpretability and simplicity, but it was ultimately unable to capture potential nonlinear boundaries in the data. In the first implementation of logistic regression for this project, the recall for class 0 was 97%, while the recall for class 1 was ~17%. On the other hand, the most effective logistic regression model in this project achieved a class 1 recall of 75%, but its precision was only around 23%. Overall, despite using model optimization techniques including threshold tuning, class balancing, and feature selection, logistic regression struggled to capture the essential relationships within the data.

Switching to a support vector machine (SVM) with a nonlinear kernel significantly improved class 1 recall and model precision. The best-performing SVM yielded a class 1 recall of ~58% and a precision of ~45%. In addition, switching from a radial basis function (RBF) kernel to a polynomial kernel of degree 2 yielded a modest but impactful improvement to the precision. To ensure fair comparison, a consistent feature set was retained across the models. The SVM's outperformance of logistic regression indicates that the increased flexibility of the SVM better captures the decision boundary and relationships in the data.

The SVM's results indicate promise for the success of more complex and computationally demanding tools in identifying quality wine, especially tree-based models, including random forests and gradient boosting machines; models better equipped to capture nonlinear relationships. On the other hand, SVM models can be further optimized with this dataset through hyperparameter tuning, keeping computational demands low while maximizing model effectiveness.